# Manoeuvring simulations
Many simulation model for ship manoeuvring have been developed in the field of ship hydrodynamics such as: the Abkowitz model <cite id="abkowitz_ship_1964">[NO_PRINTED_FORM]</cite> or the Norrbin model {cite}`norrbin_study_1960`.
This chapter will develop a general simulation model for ship manoeuvring, that can be further specified to become either the Abkowitz or Norbin model. Expressing the models on a general form is important in this research where many different models will be tested and compared.

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt
if os.name == 'nt':
    plt.style.use('book.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex, Markdown
from src.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from src.data import mdl

from src.symbols import *
from src.parameters import *
import src.symbols as symbols
from src import prime_system
from src.models import regression
from src.visualization.regression import show_pred
from src.visualization.plot import track_plot
from src.equation import Equation

## Load models:
# (Uncomment these for faster loading):
import src.models.vmm_nonlinear_EOM  as vmm 

## Ship parameters
The Ocean Bird Wind Powered Car Carrier will be used as the reference ship of this book:
![](https://www.walleniusmarine.com/wp-content/uploads/2020/09/oceanbird-960x540.jpg)

In [ ]:
ship_parameters ={'T': 0.2063106796116504,
 'L': 5.014563106796117,
 'CB': 0.45034232324249973,
 'B': 0.9466019417475728,
 'rho': 1000,
 'x_G': 0,
 'm': 441.0267843660858,
 'I_z': 693.124396594905,
 'volume': 0.4410267843660858}

ps = prime_system.PrimeSystem(**ship_parameters)  # model

A model scale version of this ship has the following main dimensions:

In [ ]:
ship_parameters

## 3 DOF
A simulation model with only three degrees of fredom (DOF) : *surge*, *sway* and *yaw* is often sufficient to describe the ship's manoeuvring performance. The equation of motion can be expressed as <cite id="2k4ro">(Triantafyllou &#38; Hover, 2003)</cite>:

In [ ]:
display(Math(vlatex(vmm.X_eom)))
display(Math(vlatex(vmm.Y_eom)))
display(Math(vlatex(vmm.N_eom)))


Where $X_{force}$, $Y_{force}$ and $N_{force}$ are the hydrodynamic forces from the ship. So these equations holds for most of the different models for manoeuvring, except linear models where the nonlinear $r\cdot u$ term have been linearized. The difference in the methods is rather in how these hydrodynamic forces are calculated.

The hydrodynamic forces can be split into added masses (that depend on the accelerations: $\dot{u}$, $\dot{v}$, $\dot{r}$) and dampings (that depend on the velocities: $u$, $v$, $r$):

In [ ]:
display(Math(vlatex(vmm.fx_eq)))
display(Math(vlatex(vmm.fy_eq)))
display(Math(vlatex(vmm.mz_eq)))

$X_{\dot{u}}$, $X_{\dot{v}}$, $X_{\dot{r}}$ are the added masses: when the ship is accelerating in water, not only the mass of the ship is accelerating, but also some mass in the water needs to be accelerated. This is referred to as added masses. Sometimes added  masses such as $X_{\dot{v}}$ is neglected, as in the case above.

The specialization of the various simulation models can now be further categorized to the functions to calculate the damping forces: $X_{qs}$, $Y_{qs}$, $N_{qs}$. The subscript "qs" stands for quasi static.

In [ ]:
ship_parameters_prime = ps.prime(ship_parameters)

The general equation for all of the simulation models in this research can be written as: 

In [ ]:
display(Math(vlatex(vmm.X_eq)))
display(Math(vlatex(vmm.Y_eq)))
display(Math(vlatex(vmm.N_eq)))

The classic simulation methods express the damping forces as a truncated taylor series. Here is a rather simple model, that has been created by the author to be used in the followin examples.

In [ ]:
display(Math(vlatex(vmm.X_qs_eq)))
display(Math(vlatex(vmm.Y_qs_eq)))
display(Math(vlatex(vmm.N_qs_eq)))

## Similitude

### Prime system
The prime system uses ship length $L$, water density $\rho$ and total velocity $U^2=u^2+v^2$ to express physical quantities in nondimensional form. The quantities have a prime symbol (') attached to them when the prime system is used. The quantities are made nondimensional with the prime system by dividing with a denominator according to the table below:

In [ ]:
prime_system.df_prime.loc[['denominator']].transpose()

The manoeuvring models are often expressed in the prime system so that a nondimensional force $F'$ for instance can be expressed as a function of some coefficients $C$ and nondimensional velocity $v$:

In [ ]:
F_,v_, C_=sp.symbols("F' v' C")
F=sp.symbols("F")

eq = sp.Eq(F_, 
      C_*v_)
eq

This may look as a model where the force depend on the transverse velocity $v$ only, but this is actually not true. If this equation is converted converted back to SI units it is instead written:

In [ ]:
eq_F_prime = sp.Eq(F,prime_system.df_prime.loc['denominator','force']*F_)
eq_v_prime = sp.Eq(v,prime_system.df_prime.loc['denominator','linear_velocity']*v_)

eqs = [eq, eq_F_prime, eq_v_prime]

sp.Eq(F,sp.solve(eqs,F,F_,v_,dict=True)[0][F])

So it is now clear that the example force model above, also depends on the total velocity $U$. 

## Brix parameters
The hydrodynamic derivatives can be estimated with semi empirical formulas according to <cite id="ycziw">(Brix, 1993)</cite>.

In [ ]:
def calculate_prime(row, ship_parameters):
    return run(function=row['brix_lambda'], inputs=ship_parameters)


mask = df_parameters['brix_lambda'].notnull()
df_parameters.loc[mask,'brix_prime'] = df_parameters.loc[mask].apply(calculate_prime, ship_parameters=ship_parameters, axis=1)

#df_parameters['brix_prime'].fillna(0, inplace=True)


In [ ]:
df_parameters['prime'] = df_parameters['brix_prime']

df_parameters.loc['Ydelta','prime'] = 0.001  # Just guessing
df_parameters.loc['Ndelta','prime'] = -df_parameters.loc['Ydelta','prime']/2  # Just guessing
#df_parameters['prime'].fillna(0, inplace=True)

In [ ]:
df_parameters.loc['Nu','prime'] = 0
df_parameters.loc['Nur','prime'] = 0
df_parameters.loc['Xdelta','prime'] = 0
df_parameters.loc['Xr','prime'] = 0
df_parameters.loc['Xrr','prime'] = 0
df_parameters.loc['Xu','prime'] = 0
df_parameters.loc['Xv','prime'] = 0
df_parameters.loc['Xvr','prime'] = 0
df_parameters.loc['Yu','prime'] = 0
df_parameters.loc['Yur','prime'] = 0

In [ ]:
mask = df_parameters['prime'].notnull()
index = df_parameters.loc[mask,'prime'].index
coefficients=vmm.simulator.get_all_coefficients(sympy_symbols=False)
missing_coefficients = set(coefficients) - set(index)
missing_coefficients

In [ ]:
mask = df_parameters['prime'].notnull()
for index, parameter in df_parameters.loc[mask].iterrows():
    display(sp.Eq(parameter['symbol'],parameter['prime']))

## Simulate data

In [ ]:
parameters=df_parameters['prime'].copy()

t_ = np.linspace(0,400,1000)
df = pd.DataFrame(index=t_)

df['u'] = 2
df['v'] = 0
df['r'] = 0
df['x0'] = 0
df['y0'] = 0
df['psi'] = 0
df['U'] = np.sqrt(df['u']**2 + df['v']**2)
df['beta'] = -np.arctan2(df['v'],df['u'])

df['delta'] = np.deg2rad(20)

result = vmm.simulator.simulate(df_=df, parameters=parameters, ship_parameters=ship_parameters, 
                                  control_keys=['delta'], primed_parameters=True,prime_system=ps)

In [ ]:
result.track_plot();

In [ ]:
result.plot()